In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # 使用cpu训练

In [31]:
mnist = np.load("../../../Other/datasets/NumPy/MNIST.npz")
x_train, y_train, x_test, y_test = mnist['x_train'], mnist['y_train'], mnist['x_test'], mnist['y_test']

x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [32]:
class MyModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    @tf.function
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [33]:
model = MyModel()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(loss)
    train_accuracy.update_state(labels, predictions)


@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss.update_state(t_loss)
    test_accuracy.update_state(labels, predictions)

In [34]:
summary_writer = tf.summary.create_file_writer("test_example0")

In [35]:
tf.summary.trace_on()
# 这里只trace模型结构
model_structure_trace = MyModel()
output = model_structure_trace(tf.random.uniform((32, 28, 28, 1)))

with summary_writer.as_default():
    tf.summary.trace_export(name="model_trace", step=0)

In [36]:
EPOCHS = 5
for epoch in range(EPOCHS):
    for (x_train, y_train) in train_ds:
        train_step(x_train, y_train)

    for (x_test, y_test) in test_ds:
        test_step(x_test, y_test)

    with summary_writer.as_default():
        tf.summary.scalar('train_loss', train_loss.result(), step=epoch)
        tf.summary.scalar('train_accuracy', train_accuracy.result(), step=epoch)
        tf.summary.scalar('test_loss', test_loss.result(), step=epoch)
        tf.summary.scalar('test_accuracy', test_accuracy.result(), step=epoch)

    print("Epoch {:d}: train_loss: {:.3f}, test_loss: {:.3f}, train_accuracy: {:.3%}, test_accuray: {:.3%}".format(
        epoch,
        train_loss.result(),
        test_loss.result(),
        train_accuracy.result(),
        test_accuracy.result()))

    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

Epoch 0: train_loss: 0.131, test_loss: 0.066, train_accuracy: 96.065%, test_accuray: 97.860%
Epoch 1: train_loss: 0.044, test_loss: 0.046, train_accuracy: 98.638%, test_accuray: 98.550%
Epoch 2: train_loss: 0.022, test_loss: 0.067, train_accuracy: 99.292%, test_accuray: 98.110%
Epoch 3: train_loss: 0.013, test_loss: 0.065, train_accuracy: 99.568%, test_accuray: 98.030%
Epoch 4: train_loss: 0.010, test_loss: 0.064, train_accuracy: 99.687%, test_accuray: 98.250%


In [ ]:
# 终端执行:tensorboard --logdir=test_example0